**Connection to Google Drive through Colab**

In [1]:
# Run this cell if you need to connect to G drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Path to project folder**

In [2]:
%cd '/content/drive/My Drive/UNSW_CyberSecurity/UNSW_SecurityEngineering/Week 5/experiment/model'

/content/drive/My Drive/UNSW_CyberSecurity/UNSW_SecurityEngineering/Week 5/experiment/model


**Packages**

In [13]:
import numpy as np
import time
import torch
import random

from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

**Define Model**

In [1]:
class ConvNet(torch.nn.Module):
    def __init__(self, hidden=64, output=10):
        super(ConvNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 4, kernel_size=7, padding=0, stride=3)
        self.fc1 = torch.nn.Linear(256, hidden)
        self.fc2 = torch.nn.Linear(hidden, output)
        self.activation = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        # the model uses the square activation function
        x = self.activation(x)
        # flattening while keeping the batch axis
        x = x.view(-1, 256)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        return x

NameError: name 'torch' is not defined

In [15]:
# Convert from numpy array to torch tensor.
def preprocessing(X: np.ndarray):
    X = torch.Tensor(X).type(torch.float32)
    return X.view(X.size(0), 1, 28, 28)

In [16]:
# Data class for DataLoader
class MNIST_dataset(Dataset):
    def __init__(self, input, ouput):
        self.input = input
        self.ouput = ouput

    def __getitem__(self, index):
        return self.input[index], self.ouput[index]

    def __len__ (self):
        return len(self.input)

**Reproducibility**


*   To ensure reproducibility as much as possible, we should set seed to each random number generators.



In [17]:
# Fix seed
seed = 1875
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

**Preprocessing**
*   Set time stamp
*   Load dataset (A subsample of 10,000 images from the MNIST dataset)
*   Normalization into [0, 1]

In [22]:
ct = time.strftime('%Y%m%d-%H%M')
path = r'../data/mnist-sampled-N10000.npz'
data = np.load(path)
data_img = data.get(data.files[0]) / 255.0 # this is input data
data_lbl = data.get(data.files[1]) # this is numeric label
N = data_img.shape[0]
p = data_img.reshape(N, -1).shape[1]

data_img = preprocessing(data_img)
data_lbl = F.one_hot(torch.Tensor(data_lbl).long(), num_classes=10).float() # One-hot key encoding
model = ConvNet()
optimizer = optim.RMSprop(list(model.parameters()), lr=1e-3)
criterion = nn.CrossEntropyLoss()



*   Prespecific the batch size and engage with DataLoader.



In [19]:
data = MNIST_dataset(data_img, data_lbl)
train_loader = DataLoader(data, batch_size=128, shuffle=True)

In [20]:
# Device: gpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if(torch.cuda.is_available()):
    model = model.cuda()

n_epochs = 200
start = time.time()
total = 0
correct = 0
for epoch in range(n_epochs):
    for i, data in enumerate(train_loader, 1):
        images, labels = data

        if(torch.cuda.is_available()):
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()
        outputs = model(images)
        total += labels.size(0)

        _, pred = torch.max(outputs.data, 1)
        _, y = torch.max(labels.data, 1)
        correct += (pred == y).sum().item()

        loss = criterion(outputs, labels)
        if(i%10 == 0):
            print('Epoch: {} Batch: {} loss: {}'.format(epoch, i, loss.item()))

        loss.backward()
        optimizer.step()

print('Training Completed in: {} secs'.format(time.time()-start))
print('Training accuracy: {} %'.format((correct/total)*100))


Epoch: 0 Batch: 10 loss: 2.299292802810669
Epoch: 0 Batch: 20 loss: 2.284409999847412
Epoch: 0 Batch: 30 loss: 2.2656595706939697
Epoch: 0 Batch: 40 loss: 2.2261059284210205
Epoch: 0 Batch: 50 loss: 2.179088592529297
Epoch: 0 Batch: 60 loss: 2.1483407020568848
Epoch: 0 Batch: 70 loss: 2.105125904083252
Epoch: 1 Batch: 10 loss: 2.033027172088623
Epoch: 1 Batch: 20 loss: 2.005035638809204
Epoch: 1 Batch: 30 loss: 1.9876081943511963
Epoch: 1 Batch: 40 loss: 1.9670908451080322
Epoch: 1 Batch: 50 loss: 1.940164566040039
Epoch: 1 Batch: 60 loss: 1.9247775077819824
Epoch: 1 Batch: 70 loss: 1.8934404850006104
Epoch: 2 Batch: 10 loss: 1.8663971424102783
Epoch: 2 Batch: 20 loss: 1.8770167827606201
Epoch: 2 Batch: 30 loss: 1.8736019134521484
Epoch: 2 Batch: 40 loss: 1.8273463249206543
Epoch: 2 Batch: 50 loss: 1.8032853603363037
Epoch: 2 Batch: 60 loss: 1.830317735671997
Epoch: 2 Batch: 70 loss: 1.7649418115615845
Epoch: 3 Batch: 10 loss: 1.8223683834075928
Epoch: 3 Batch: 20 loss: 1.7611198425292

In [21]:
torch.save(model.state_dict(), "./model.pth")